In [1]:
%matplotlib inline
import sys
import math
import time
import random
import collections
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from Polymer import Polymer
from copy import deepcopy
import scipy.io as sio
import os.path

In [19]:
def recursiveGrowth(pol):
    global polarray
    global max_polymer_size
    global polymer_count
    if (len(polarray) >= polymer_count):
        return
    weight = pol.addBead()
    avweight = getAverageWeight();
    upLim = 2 * avweight / pol.weight3
    lowLim = 1.2 * avweight / pol.weight3
    if (pol.size >= max_polymer_size):
        return
    if (pol.weight > upLim):
        pol.weight = pol.weight / 2;
        tpol = deepcopy(pol)
        polarray.append(tpol)
        if (len(polarray)%1000 == 0):
            print("Simulation progress: " + str(int(len(polarray)*100/polymer_count)) + "%") 
            sys.stdout.flush()
        recursiveGrowth(pol)
        recursiveGrowth(tpol)
    if (pol.weight < lowLim):
        rand = random.uniform(0,1)
        if (rand < 0.5):
            pol.weight = pol.weight * 2
            recursiveGrowth(pol)
        else:
            pol.alive = False;

def getAverageWeight():
    global polarray;
    weight_count = 0
    weight = 0
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            weight_count = weight_count + 1
            weight += polarray[i].weight
    avgweight = weight / weight_count
    return avgweight

def Grow(polarray):  # Add a bead to all living polymers
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            polarray[i].addBead()
    return polarray

def Prune(polarray):  # Prune the polymers on their average weight
    # Calculate the average weight of the living polymers
    weight = 0
    alive = 0
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            weight += polarray[i].weight
            alive += 1
    weight = weight / alive;
    print("Alive: "+str(alive) + "Weight: "+str(weight))
    # Prune the polymers on their weight
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            if (polarray[i].weight > 2 * weight / polarray[i].weight3): # Branch
                polarray[i].weight /= 2;
                polarray.append(deepcopy(polarray[i]))
            if (polarray[i].weight < 2 * weight / polarray[i].weight3): # Prune
                rand = random.uniform(0,1)
                if (rand < 0.5):
                    polarray[i].weight *= 2
                else:
                    polarray[i].alive = False;
    return polarray

In [ ]:
bead_position = np.zeros([2,2])       # Initialize array
bead_position[1,:] = [1,0]            # Set initial condition
polymer_size = 2                      # Amount of beads in the polymer
max_polymer_size = 250                # Maximum polymer size
polymer_count = 1E5                   # Amount of polymers that we are simulating

T = 5                                 # Temperature in kelvin
epsilon = 0.25                        # Epsilon voor de lennard Jones
sigma = 0.8                           # Sigma voor de Lennard Jones

theta = 6                             # Number of angles
theta_weight = np.zeros([theta,1])    # Weight of angles
theta_weight_sum = 0                  # Sum of the weights
polymer_weight = 1                    # Weight

polarray = []                         # List containing all the polymers

# We start with 5 polymers.
for i in range(0,2):
    pol = Polymer(bead_position, polymer_weight, 2, theta, epsilon, sigma, T)
    polarray.append(pol)

for l in range (2,250):
    polarray = Grow(polarray)
    polarray = Prune(polarray)
    print("Polymer length " + str(len(polarray)) + "  Size = "+str(l)) 
    sys.stdout.flush()
print("Done")

Alive: 2Weight: [ 4.59929559]
Polymer length 4  Size = 2
Alive: 2Weight: [ 11.2003308]
Polymer length 5  Size = 3
Alive: 2Weight: [ 45.64369396]
Polymer length 6  Size = 4
Alive: 2Weight: [ 201.07104749]
Polymer length 7  Size = 5
Alive: 3Weight: [ 805.65090335]
Polymer length 8  Size = 6
Alive: 3Weight: [ 3380.42227777]
Polymer length 11  Size = 7
Alive: 4Weight: [ 7995.51503918]
Polymer length 13  Size = 8
Alive: 4Weight: [ 27473.00859973]
Polymer length 16  Size = 9
Alive: 5Weight: [ 91090.15573138]
Polymer length 20  Size = 10
Alive: 8Weight: [ 291062.85732527]
Polymer length 24  Size = 11
Alive: 8Weight: [ 1011428.35284655]
Polymer length 28  Size = 12
Alive: 10Weight: [ 3332559.10372986]
Polymer length 35  Size = 13
Alive: 13Weight: [ 11282270.70359323]
Polymer length 44  Size = 14
Alive: 17Weight: [ 38380459.68481172]
Polymer length 55  Size = 15
Alive: 21Weight: [  1.51188211e+08]
Polymer length 68  Size = 16
Alive: 24Weight: [  5.86568675e+08]
Polymer length 83  Size = 17
Aliv

In [21]:
print(len(polarray))
dist = np.zeros([max_polymer_size+1, 3])
positions = np.zeros([max_polymer_size+1, len(polarray),2])
for i in range(2, max_polymer_size+1):
    dist[i,0] = i;
    dis = 0
    c = 0
    for j in range(0, len(polarray)):
        if (polarray[j].size >=i):
            dis += polarray[j].getLength(i)
            positions[i,j,:] = polarray[j].getPosition(i-1)
            c = c + 1
    if (c > 0):
        dist[i,1] = (dis / c)
    else:
        dist[i,1] = 0
    dist[i,2] = dist[i,0] ** 0.75
name_i = 0;
while (os.path.isfile('DataPoints/data_file_'+str(name_i)+'.mat') != False):
    name_i = name_i + 1
sio.savemat('DataPoints/data_file_'+str(name_i)+'.mat', {'Distances':dist, 'Positions':positions})

4123
